
## P7: Résolvez des problèmes en utilisant des algorithmes en Python #3


Dans le 2nd jeu d'essai de la mission, il y a 956 actions ; l'explosion combinatoire rend impossible le traitement car il existe 6.090821257124996 e+287 combinaisons.  

Le nombre de combinaison est la somme du nombre de combinaisons de k pris parmi n,  k allant de 1 à n.
C'est n fois le nombre de combinaisons.
$$
n \times {n \choose k}= n \times {\frac {n!}{k!(n-k)!}}  
$$

Il faut donc procéder autrement.  
L'algorithme du sac à dos où l'on parcourt un arbre des cas possibles en décidant de prendre ou non une action peut être optimisé.  

  

### 1. Préparation des données



In [1]:
# modules importés
# lecture de fichiers csv et dict colonnes
import csv as csv
# nettoyage des caractères via expression regex
import re as re
# mesure du temps passé -> time spend over complexity
import time
# mesure de l'occupation mémoire space complexity
from sys import getsizeof


In [2]:
# if used does approximate a function to Big_O models
if False:
    try:
        import big_o
    except ModuleNotFoundError:
        print('cherche',ModuleNotFoundError)
        !pip install --upgrade pip
        !pip install big_o
        import big_o

In [3]:
# constants
FILE = "data/dataset2_Python+P7.csv" 
FIELDNAMES = ['name', 'cost', 'profit']
STEP = 100 # 100 times de value
BUDGET = 500 * STEP

In [4]:
 
def fn_timer(function):
    """ starts before & stops after the run of the function, a time counter"""        
#    @wraps(function)
    def function_timer(*args, **kwargs):
        t0 = time.perf_counter_ns()
        result = function(*args, **kwargs)
        t1 = time.perf_counter_ns()
        elapsed = (t1-t0)/1000000000
        print(f"Total time running {function.__name__}: {str(elapsed)}s seconds")
        return result
    return function_timer

In [5]:
# strips a string from its weird caracters
def clean_char(texte: str) -> str:
    """ on ne conserve que les caractères lisibles 
    les lettres, chiffres, ponctuations décimales et signes
    les valeurs negatives sont acceptées, du point de vue profit.
    """
    texte_propre = re.sub(r"[^a-zA-Z0-9\-\.\,\+]", "", texte.replace(',','.'))
    return texte_propre

In [6]:
# get the memory allocation for int & float
print('python size allocated to an integer is :', getsizeof(int(13000000)),
      'whereas size to float is :', getsizeof(float(13000000.05)))

python size allocated to an integer is : 28 whereas size to float is : 24


In [7]:
@fn_timer
def measure_walk_thru(liste_numerique: list):
    compte = 0
    for index,valeur in enumerate(liste_numerique):
        compte +=valeur
    print(compte)
#liste_entier_millions = [int(x) for x in range(100000000)]
#liste_flottant_millions = [float(x/2) for x in range(100000000)]

In [8]:
#measure_walk_thru(liste_entier_millions)

In [9]:
#measure_walk_thru(liste_flottant_millions)

### Le choix du type float ou entier pour les couts et profits a t-il un impact significatif sur l'efficacité de nos algorithme avec cette implémentation de Python?

Au niveau de l'allocation mémoire, une comparaison de la mémoire allouée avec sys.getsizeof, nous donne :
python size allocated to an integer is : 28 whereas size to float is : 24

Au niveau du temps de traitement, une comparaison ne montre pas de diffrérence notable pour 100 milions d'éléments en liste. 
measure_walk_thru(liste_entier_millions)
Total time running measure_walk_thru: 26.040572s seconds
measure_walk_thru(liste_flottant_millions)
Total time running measure_walk_thru: 30.3793064s seconds

:ref:
[15. Floating Point Arithmetic: Issues and Limitations — Python 3.10.2 documentation](https://docs.python.org/3/tutorial/floatingpoint.html)
Unfortunately, most decimal fractions cannot be represented exactly as binary fractions. A consequence is that, in general, the decimal floating-point numbers you enter are only approximated by the binary floating-point numbers actually stored in the machine.

L'algorithme de programmation dynamique utilisé ici (Knapsack ou sac à dos) faire intervenir une matrice de longueur le nombre d'action (soit 957 dans le 1er jeu de test retenu) et de largeur le budget alloué. Les index de déplacement dans ce tableau doivent être des entiers.
En multipliant par 100 les montants, nous obtiendrons une plus grande précision des calculs au détriment d'un temps plus long en algo sac à dos (la taile de la matrice est de 957 nb actions x 50000 incréments de budget).

C'est un arbitrage par rapport à la demande. S'agissant d'un cabinet financier ils ont besoin de plus de précision et ont les moyens d'acheter un PC plus rapide ;-)



### 2. Amélioration des données


#### 2.1 Nettoyage technique.
On observe les données reçues pour s'assurer de leur qualité.
Cas particuliers traités:
Cout nul ; il pourrait s'agir d'action gratuite mais nous avons décidé de les supprimer du jeu d'essai.
Cout negatif ; supprimé.
profit négatif ; conservé car une perte est possible.

On aurait pu diminuer encore plus le nombre d'action en suppromant les actions dont le profit =0.

#### 2.2 Nettoyage métier.
On vérifie que le profit est possiblement un pourcentage du cout et non une valeur.
La bourse rapporte 8% en moyenne par an sur les 30 dernières années.

:ref:
[Palmarès des placements sur trente ans : le triomphe de l’investissement en actions | Le Revenu](https://www.lerevenu.com/placements/palmares-des-placements-sur-trente-ans-le-triomphe-de-linvestissement-en-actions)

Entre 1988 et 2018 (chiffres arrêtés au 18 mai), les actions avec dividendes ont rapporté 1.352%, l’immobilier à Paris 402%, l’assurance vie en euros370%, les Sicav monétaires 209%, l’or 179%, le Livret A 135%. À titre d’information, l’inflation sur la période a été de 67%.
Hors dividendes, la performance tombe à 461%, ce qui reste très honorable. 
-> Prenons un ratio de 15%/an.

Si plus de 50% des actions ont un ratio profit/cout >> 1,30 alors on estime que le profit est estimé en valeur.

Après estimation via Excel, 691 actions au profit < 1,30 et donc 266 > 1.30.
On estime que le profit est exprimé en % age du coût.

=> Finalement ne pas retirer les petites valeurs de cout qui permettent peut-être de grapiller un peu de profit aux derniers Eur de budget restant.  

In [10]:
""" lecture, nettoyage et chargement en dict.
    les non valeurs NaN sont rejetées.
"""
action_dict = {}
try:
    with open(FILE, "r", newline='', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file, fieldnames=FIELDNAMES, 
                                    delimiter=',', doublequote=False)
        # skip the header
        next(csv_reader)
        compteur_ligne = 0
        for idx, line in enumerate(csv_reader):
            clean_data = True

            if line[FIELDNAMES[0]] != "":
                cle = clean_char(line[FIELDNAMES[0]])
            else:
                print(f" line {idx} had missing share name; dropped.")
                clean_data = False

            if line[FIELDNAMES[1]] != "":
                cout = int(STEP * float(clean_char(line[FIELDNAMES[1]])))
                if cout < 0 :
                    print(f" line {idx} had neg cost data; dropped.")
                    cout = 0
                    clean_data = False                    
                if cout == 0 :
                    print(f" line {idx} had null cost data ; could have been a gift but management decision: dropped.")
                    cout = 0
                    clean_data = False                                        
            else:
                print(f" line {idx} had missing cost data; dropped.")
                clean_data = False
                
            if line[FIELDNAMES[2]] != "":
                gain_percent = int(STEP * float(clean_char(line[FIELDNAMES[2]])))
            else:
                print(f" line {idx} had missing profit percentage; dropped.")
                clean_data = False
            if gain_percent <= 0:
                # TODO: check if to keep or not in any case comment ; as negativ can't be optimum
                print(f"** line {idx} had negative profit percentage ; accepted but pls check. **")
                print('      ',idx,line)                
            if clean_data:
                action_dict[cle] = (cout, cout*gain_percent/STEP)
                compteur_ligne += 1
            else:
                print('      ',idx,line)    
        print("nombre d'actions retenues: ", compteur_ligne)
except FileNotFoundError:
    print(f" fichier non trouvé, Merci de vérifier son nom {file_name} : {FileNotFoundError}")            
except IOError:
    print(f" une erreur est survenue à l'écriture du fichier {file_name} : {IOError}")      


 line 2 had neg cost data; dropped.
 line 2 had null cost data ; could have been a gift but management decision: dropped.
2 {'name': 'Share-AXWK', 'cost': '-9.27', 'profit': '27.19'}
 line 7 had neg cost data; dropped.
 line 7 had null cost data ; could have been a gift but management decision: dropped.
7 {'name': 'Share-FDLS', 'cost': '-6.38', 'profit': '10.96'}
 line 8 had neg cost data; dropped.
 line 8 had null cost data ; could have been a gift but management decision: dropped.
8 {'name': 'Share-JDYS', 'cost': '-2.7', 'profit': '11.96'}
 line 12 had neg cost data; dropped.
 line 12 had null cost data ; could have been a gift but management decision: dropped.
12 {'name': 'Share-RPPH', 'cost': '-2.32', 'profit': '38.98'}
 line 17 had neg cost data; dropped.
 line 17 had null cost data ; could have been a gift but management decision: dropped.
17 {'name': 'Share-ULRO', 'cost': '-5.34', 'profit': '19.83'}
 line 19 had neg cost data; dropped.
 line 19 had null cost data ; could have be

In [11]:
#action_dict
                

### 3. Résolution en force brute récurrente (pour mémoire):
si on considère l'ensemble des 'actions' du porte-feuille potentiel, il existe de multiples combinaisons d''action'. Parmi toutes les combinaisons dont le cout est inférieur au budget d'investissement, l'une d'entre elles est optimale car elle fournit le plus grand profit.


Maintenant quand on prend une 'action' au hasard,
soit on selectionne cette 'action' comme partie de la solution
soit on ne la selectionne pas.

Quand on selectionne une 'action', il faut exprimer la valeur et le cout du porte-feuille en fonction de sa valeur avant sélection (pour introduire de la récurrence avec n fonction de n-1).
valeur(pf(i)) = valeur(pf(i-1)) + valeur(action(i))
cout(pf(i)) = cout(pf(i-1)) + cout(action(i))
ou avec un cout exprimé en budget restant:
budget_restant(pf(i)) = budget_restant(pf(i-1)) - cout(action(i))

Quand on ne selectionne pas une action, la valeur et le poids du porte-feuille sont inchangés.

Cas d'arrêt de la fonction récurrente : si tout le budget est épuisé ou si toutes les actions ont été considérées.

Formalisons un peu mieux:

In [12]:
# recursively check all combinations

def knap_sack_brute(budget, actions, actions_porte_feuille=[]):
    # tant qu'il reste des actions non traitées
    if actions:
        # si l'action n'était pas retenue
        profit_sans, liste_sans = knap_sack_brute(budget, actions[1:], actions_porte_feuille)
        # 1ère action du porte-feuille
        val = actions[0]
        cout = val[1]
        # si son cout est inférieur au budget
        if cout <= budget:
            # action prise, budget diminuée de son cout, et sous ensemble 
            #   d'action restante et selection de l'action courante 
            profit_avec, liste_avec = knap_sack_brute(budget - cout, actions[1:], actions_porte_feuille + [val])
            # Choix de l'optimum :
            if profit_sans < profit_avec:
                return profit_avec, liste_avec

        return profit_sans, liste_sans
    else:
        return sum([i[2] for i in actions_porte_feuille]), actions_porte_feuille
# This code was inspired by Algomius

In [13]:
if False:
    liste_actions = [(cle,val[0],val[1]) for cle,val in action_dict.items()]
    valeur, action_pf = knap_sack_brute(BUDGET,liste_actions, [])
    print('Un profit de ',valeur, ' pour la selection d\'action ', action_pf)

### 4. Résolution en programmation dynamique

L'idée est de mémoriser les étapes déja calculées et de choisir pour un niveau de budget l'optimum entre la nouvelle action du porte-feuille et la meilleure combinaison déja mémorisées pour le reliquat de bugdet.

In [14]:
def knap_sack_optimise(budget, actions):
    # initialize matrix of size budget x share number
    # columns are budget increasing value whereas every line is a share
    matrice = [[0 for x in range(0, budget + 1)] for x in range(len(actions) + 1)]
    # for every share in book ; +1 as the zero line 
    for i in range(1, len(actions) + 1):
        # for every budget value up to max BUDGET
        for cout_courant in range(1, budget + 1):
            # if enough money to buy the current share
            if actions[i-1][1] <= cout_courant:
                # take the max btw 
                # current share profit + the previous best solution for remaining cost btw courrent_budget - share cost 
                # the previous Max value of the same budget column
                matrice[i][cout_courant] = max(actions[i-1][2] + matrice[i-1][cout_courant-actions[i-1][1]], matrice[i-1][cout_courant])
            else:
            # if not enough money, keep the previous solution
                matrice[i][cout_courant] = matrice[i-1][cout_courant]

    # track back the selected shares
    w = budget
    n = len(actions)
    actions_porte_feuille = []

    while w >= 0 and n >= 0:
        e = actions[n-1]
        if matrice[n][w] == matrice[n-1][w-e[1]] + e[2]:
            actions_porte_feuille.append(e)
            w -= e[1]

        n -= 1

    return matrice[-1][-1], actions_porte_feuille
# This code also was inspired by Algomius

In [15]:
if True:
    liste_actions = [(cle,val[0],val[1]) for cle,val in action_dict.items()]
    valeur, action_pf = knap_sack_optimise(BUDGET,liste_actions)
    print('Un profit de ',round(valeur/100), ' pour la selection d\'action ', action_pf)

Un profit de  19795  pour la selection d'action  [('Share-ECAQ', 3166, 125025.34), ('Share-IXCI', 2632, 103700.8), ('Share-FWBE', 1830, 72870.6), ('Share-ZOFA', 2532, 100722.96), ('Share-PLLK', 1994, 79560.6), ('Share-LXZU', 424, 16764.96), ('Share-YFVZ', 2255, 88170.5), ('Share-ANFX', 3854, 153080.88), ('Share-PATS', 2770, 110716.9), ('Share-SCWM', 642, 24460.2), ('Share-NDKR', 3306, 131909.4), ('Share-ALIY', 2908, 116116.44), ('Share-JWGF', 4869, 194419.17), ('Share-JGTW', 3529, 139148.47), ('Share-FAPS', 3257, 128781.78), ('Share-VCAX', 2742, 106910.58), ('Share-LFXB', 1483, 59008.57), ('Share-DWSK', 2949, 116043.15), ('Share-XQII', 1342, 53022.42), ('Share-ROOM', 1506, 59065.32)]


In [24]:
print('Un profit de ',round(valeur/(STEP*STEP),2), ' pour la selection d\'action ', [x[0] for x in action_pf])

Un profit de  197.95  pour la selection d'action  ['Share-ALIY', 'Share-ANFX', 'Share-DWSK', 'Share-ECAQ', 'Share-FAPS', 'Share-FWBE', 'Share-IXCI', 'Share-JGTW', 'Share-JWGF', 'Share-LFXB', 'Share-LXZU', 'Share-NDKR', 'Share-PATS', 'Share-PLLK', 'Share-ROOM', 'Share-SCWM', 'Share-VCAX', 'Share-XQII', 'Share-YFVZ', 'Share-ZOFA']


In [17]:
print('Un cout de ', sum([x[1] for x in action_pf])/STEP)

Un cout de  499.9


In [18]:
@fn_timer
def measure_optimise(budget, liste):
    return knap_sack_optimise(budget,liste)
@fn_timer
def measure_brute(budget, liste):
    return knap_sack_brute(budget,liste,[])

In [19]:
#liste_actions = [(cle,val[0],val[1]) for cle,val in action_dict.items()]
#measure_optimise(BUDGET,liste_actions)


In [20]:
#liste_actions = [(cle,val[0],val[1]) for cle,val in action_dict.items()]
#measure_brute(BUDGET,liste_actions)